# DROP TABLE statement

```postgresql
DROP TABLE [IF EXISTS] table_name
[CASCADE | RESTRICT]; 
```

* If you remove a table that does not exist, PostgreSQL issues an error. 
* To avoid the error, you can use the `IF EXISTS` option.

If the table is used in other database objects such as views, triggers, functions, and stored procedures, you cannot remove it. In this case, you have two options:
* Use the `CASCADE` option to remove the table and its dependent objects.
* Use the `RESTRICT` option to reject the removal if there is any object depending on the table. The `RESTRICT` option is the default if you don’t explicitly specify it in the `DROP TABLE` statement.

To remove multiple tables simultaneously, you can place the tables separated by commas after the `DROP TABLE` keywords:
```postgresql
DROP TABLE [IF EXISTS]
   table_name_1,
   table_name_2,
   ...
[CASCADE | RESTRICT];
```

> Note that you need to have the roles of the **superuser**, **schema owner**, or **table owner** to **drop tables**.

**Drop a table that does not exist**
```postgresql
DROP TABLE author;   -

[Err] ERROR:  table "author" does not exist
```

To avoid the error, you can use the `IF EXISTS` option.
```postgresql
DROP TABLE IF EXISTS author;

NOTICE:  table "author" does not exist, skipping DROP TABLE
```
The output indicates that PostgreSQL issued a notice instead of an error.

**Drop a table that has dependent objects**
```postgresql
DROP TABLE IF EXISTS authors; 
```

Because the author's table has a dependent object, which is a foreign key that references the pages table, PostgreSQL issues an error message:
```postgresql
ERROR:  cannot drop table authors because other objects depend on it
DETAIL:  constraint pages_author_id_fkey on table pages depends on table authors
HINT:  Use DROP ... CASCADE to drop the dependent objects too.
```

In this case, you need to remove all dependent objects first before dropping the author table, or use the `CASCADE` option as follows:
```postgresql
DROP TABLE authors CASCADE;
```
This statement deletes the **authors** table as well as the constraint in the **pages** table.

If the `DROP TABLE` statement removes the dependent objects of the table that are being dropped, it will issue a notice like this:
```postgresql
NOTICE:  drop cascades to constraint pages_author_id_fkey on table pages
DROP TABLE
```

**Drop multiple tables**
```postgresql
DROP TABLE tvshows, animes;
```

# TRUNCATE TABLE statement

* To remove all data from a table, you use the `DELETE` statement without a `WHERE` clause. 
* However, when the table has numerous data, the `DELETE` statement is not efficient.
* In this case, you can use the `TRUNCATE TABLE` statement.
* The `TRUNCATE TABLE` statement deletes all data from a table very fast.

```postgresql
TRUNCATE TABLE table_name; 
```

**Remove all data from multiple tables**
```postgresql
TRUNCATE TABLE
    table_name1,
    table_name2,
    ...; 
```

**Remove all data from a table that has foreign key references**
* In practice, the table you want to delete all data from often has foreign key references from other tables.
* By default, the `TRUNCATE TABLE` statement does not remove any data from the table that has foreign key references.
* Note that the `TRUNCATE TABLE` statement uses the `RESTRICT` option by default to prevent a table that is referenced by a foreign key from being truncated.

To remove data from a table and other tables that have foreign key references to the table, you use the `CASCADE` option in the `TRUNCATE TABLE` statement as follows :

```postgresql
TRUNCATE TABLE table_name
CASCADE; 
```

## Restarting sequence

Besides removing data, you may want to reset the values of the identity column by using the `RESTART IDENTITY` option like this:
```postgresql
TRUNCATE TABLE table_name
RESTART IDENTITY; 
```
* By default, the `TRUNCATE TABLE` statement uses the `CONTINUE IDENTITY` option.
* This option does not restart the value in the sequence associated with the column in the table.

## TRUNCATE TABLE statement and ON DELETE trigger

* Even though the `TRUNCATE TABLE` statement removes all data from a table, it does not fire any `ON DELETE` triggers associated with the table.
* To fire the trigger when the `TRUNCATE TABLE` statement executes, you need to define  `BEFORE TRUNCATE` and/or  `AFTER TRUNCATE` triggers for that table.

## TRUNCATE TABLE statement and transactions

The `TRUNCATE TABLE` is transaction-safe, meaning that you can place it within a transaction.

## Why TRUNCATE TABLE statement is more efficient than the DELETE statement

The `TRUNCATE TABLE` statement is more efficient than the `DELETE` statement due to the following main reasons:

**Minimal logging**:
* The `TRUNCATE TABLE` statement doesn’t generate individual row deletion logs.
* Instead, it deallocates entire data pages, making it faster than the `DELETE` statement.

**Fewer resources**: 
* The truncate operation is more lightweight than the delete option because it doesn’t generate as much undo and redo information.
* It releases storage space without scanning individual rows.

**Lower-level locking mechanism**: 
* The truncate operation often requires lower-level locks and is less prone to conflicts with other transactions, which improves overall system concurrency.

# Temporary Table

* In PostgreSQL, a temporary table is a table that exists only during a database session. 
* It is created and used within a single database session and is automatically dropped at the end of the session.

**Creating a temporary table**
```postgresql
CREATE TEMPORARY TABLE table_name(
   column1 datatype(size) constraint,
   column1 datatype(size) constraint,
   ...,
   table_constraints
); 
```

The `TEMP` and `TEMPORARY` keywords are equivalent so you can use them interchangeably:
```postgresql
CREATE TEMP TABLE table_name(
   ...
); 
```

* If you open a second database session and query data from the **temporary table**, you’ll get an error.
* If you terminate the current database session and attempt to query data from the mytemp table, you’ll encounter an error.
* This is because the temporary table was dropped when the session that created it ended.

**Temporary table names**
* A temporary table can have the same name as a permanent table, even though it is not recommended.
* When you create a temporary table that shares the same name as a permanent table, you cannot access the permanent table until the temporary table is removed.

**Removing a temporary table**

To drop a temporary table, you use the `DROP TABLE` statement. 
```postgresql
DROP TABLE customers;
```

Unlike the `CREATE TABLE` statement, the `DROP TABLE` statement does not have the `TEMP` or `TEMPORARY` keyword created specifically for temporary tables.

**When to use temporary tables**

**Isolation of data**: 
* Since the temporary tables are session-specific, different sessions or transactions can use the same table name for temporary tables without causing a conflict.
* This allows you to isolate data for a specific task or session.

**Intermediate storage**: 
* Temporary tables can be useful for storing the intermediate results of a complex query.
* For example, you can break down a complex query into multiple simple ones and use temporary tables as intermediate storage for storing the partial results.

**Transaction scope**: 
* Temporary tables can also be useful if you want to store intermediate results within a transaction.
* In this case, the temporary tables will be visible only to that transaction.